In [6]:
import os
import pandas as pd

# Path to the directory containing the CSV files
directory_path = '/data1-8tb-blk/sp2023stock/Insight-Wave/TopicModeling/result/integrationOutput/id/thunhoon_ids.csv'

articles_list = {}

df = pd.read_csv(directory_path)
articles_list = df['ID'].tolist()

print(f'max = {max(articles_list)}, min ={min(articles_list)}')


max = 285558, min =265812


In [7]:
import os
main = os.getcwd()
dataset_thunhoon = main + '/../../../data/thunhoon/article/'

start_process = main +  '/../../../src/Parsing/thunhoon/parsing_progress.txt'
end_process = main + '/../../../src/Crawling/thunhoon/progress.txt'

In [8]:
import os
import csv
import re
import pandas as pd
import chardet
import datetime
from bs4 import BeautifulSoup


def htmlInput(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    return soup


#Extract title
def extract_title(soup):
    title = soup.title.string.strip() if soup.title else None
    return title

#Extract body
def extract_body(soup):
    article_divs = soup.find_all("div", class_="article-p")
    articles = []
    if article_divs:
        for article_div in article_divs:
            for p in article_div.find_all('p', {'style': 'text-align:right'}):  # if <div> contains <p style="text-align:right">
                p.decompose()  # remove <p> tag
            articles.append(article_div.get_text(strip=True))
    
        articles = [s.replace('\n', ' ') for s in articles]
        articles = [s.replace('\xa0', ' ') for s in articles]
        articles = [item for item in articles if item != '']

    
    return articles


#Extract tags
def extract_tags(soup):
    tagcloud_span = soup.find("h6", class_="newsTagColor")
    
    if tagcloud_span is not None:
        tags = tagcloud_span.find_all('a')
        tag_content = [tag.get_text(strip=True) for tag in tags]
        tag_content = [item for item in tag_content if item != '']
    else:
        tag_content = None
    
    return tag_content

def extract_pubdate(soup):
    month_list = {
        'มกราคม': '01', 
        'กุมภาพันธ์': '02', 
        'มีนาคม': '03', 
        'เมษายน': '04', 
        'พฤษภาคม': '05', 
        'มิถุนายน': '06', 
        'กรกฎาคม': '07', 
        'สิงหาคม': '08', 
        'กันยายน': '09', 
        'ตุลาคม': '10', 
        'พฤศจิกายน': '11', 
        'ธันวาคม': '12'
    }
    
    # Assuming soup is already defined with your HTML content
    date_time_div = soup.find("p", class_=re.compile("^Article__PublishTimeStamp-sc-18ssv56-0"))
    
    if date_time_div is not None:
        date_time_string = date_time_div.get_text(strip=True)
    
        # Split the string into words
        words = date_time_string.split()
    
        # Identify the components
        day = words[0]
        month = month_list.get(words[1])  # Get the numeric month
        year = words[2]
        time = words[4]
    
        # Form the date and time in the desired format
        pubdate = f"{year}-{month}-{day} {time}"
    else:
        pubdate = None
    
    return pubdate

#Extract title
def extract_intro(soup):
    intro_div = soup.find("h2", class_="entry-sub-title")
    intro = ""
    if intro_div:
        intro = intro_div.get_text(strip=True)
    else:
        intro = None

    return intro

def extract_urlPic(soup):
    picture_div = soup.find('div', class_="col-sm-12")
    if picture_div:
        picture = picture_div.find('img')
        if picture:
            url = picture.get('src')  # First try to get the 'src' attribute
            if not url:  # If 'src' is None, try to get the 'data-cfsrc' attribute
                url = picture.get('data-cfsrc')
            if url:  # If we have a URL, print it
                print(f"Image URL: {url}")
            else:  # If both 'src' and 'data-cfsrc' are missing or None
                print("No src or data-cfsrc attribute found for the image.")
        else:
            print("No img tag found inside the div.")
            url = None
    else:
        print("No div with class 'col-sm-12' found.")
        url = None
    
    return url

# #Start
# if os.path.exists(start_process):
#     with open('parsing_progress.txt','r') as f:
#         progress_start = int(f.readline().strip())
#         start_id = progress_start - 1
# else: 
#     print("Start path doesn't exists process start from 1")
#     start_id = 1 
    
# article_id = start_id

# #End
# if os.path.exists(end_process):
#     with open(process,'r') as f:
#         progress_end = int(f.readline().strip())
#         end_id = progress_end + 1
# else: 
#     print("End path doesn't exists")
# print(f'Start parsing at {start_id} and Ending at {end_id}' )


print(f'Start parsing process')
for article_id in articles_list: #range(start_id, end_id+1):
    file_path = dataset_thunhoon + str(article_id) + "/index.txt"
    print(f'Processing article id {article_id}')

    if os.path.exists(file_path):
        try:
            soup = htmlInput(file_path)
            title = extract_title(soup)
            intro = extract_intro(soup)
            articles = extract_body(soup)
            pubdate = extract_pubdate(soup)
            tag_content = extract_tags(soup)
            url = extract_urlPic(soup)
            
            data_dict = {'Title': title, 'Intro': intro, 'Article': articles, 'DateTime': pubdate, 'Tags': tag_content, 'url_picture': url}
            
            output_path = dataset_thunhoon+ str(article_id) + "/parsing.txt"
            with open(os.path.join(output_path), 'w', encoding="utf-8") as f:
                for key, value in data_dict.items():
                    if value is not None:
                        if key == 'Tags' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for tags in value:
                                f.write(f"{tags}\n")

                        elif key == 'Article' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for bodys in value:
                                if bodys == 'SPONSORED':
                                    continue
                                else:
                                    f.write(f"{bodys}\n")
                                    f.write("\n")
                        else:
                            f.write(f"[::{key}::]\n{value}\n")


            with open('parsing_progress.txt', 'w') as f:
                f.write(f'{article_id}')

            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Valid'
                writer.writerow([article_id, validation])

                    
        except Exception as e:
            print(f"Error processing article {article_id}: {e}")
            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Error'
                writer.writerow([article_id, validation])
                continue


# Write the last processed article_id to 'csv_progress.txt' after loop ends
with open('parsing_progress.txt', 'w') as f:
    f.write(str(article_id) + '\n')

Start parsing process
Processing article id 285558
Image URL: https://thunhoon.com/cache/image/article/653957/EP-เล็งผลงาน-26-12-66.jpeg
Processing article id 285557
Image URL: https://thunhoon.com/cache/image/article/653956/set-p3.jpeg
Processing article id 285556
Image URL: https://thunhoon.com/cache/image/article/653955/set-p1.jpeg
Processing article id 285555
Image URL: https://thunhoon.com/cache/image/article/653948/32B-27122023.jpeg
Processing article id 285554
Image URL: https://thunhoon.com/cache/image/article/653943/กสิกรไทย-28122023.jpeg
Processing article id 285553
Image URL: https://thunhoon.com/cache/image/article/653936/32A-27122023.jpeg
Processing article id 285552
Image URL: https://thunhoon.com/cache/image/article/653969/3D-PSTC-28122023.jpeg
Processing article id 285551
Image URL: https://thunhoon.com/cache/image/article/653885/TSE-28122023.jpeg
Processing article id 285550
Image URL: https://thunhoon.com/cache/image/article/653879/สรุปมูลค่าการซื้อขายสิ้นวัน_27122023